<a href="https://colab.research.google.com/github/miladmirzazadeh/Gamein-Prototype/blob/master/prototype_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



TESTING CODES ############################################################################



In [211]:
Controller.create_player("raman")

In [212]:
Controller.create_player("ali") 


In [213]:
Controller.player_objects

{'ali': <__main__.Player at 0x7f6d6a8c2668>,
 'raman': <__main__.Player at 0x7f6d6a907cc0>}

In [218]:
Controller.put_into_market("ali", "corn", 150, 200)

'Done :)'

In [215]:
player_obj = Controller.player_objects["ali"]
stockhouse = list(player_obj.land_objects.values())[0].stockhouse


In [216]:
Controller.start_production("ali","corn", 1100)

In [217]:
stockhouse.inventory_df

,total_inventory,inhand_inventory
corn,1100,1100


In [222]:
list(Market.product_on_vitrin_objects.values())[0].player_obj

AttributeError: ignored

In [220]:
for i in list(Market.product_on_vitrin_objects.values()):
  print(i.player_obj.name)

AttributeError: ignored

In [207]:
list(Controller.player_objects["milad"].land_objects.values())[0]. machine_objects

KeyError: ignored

In [ ]:
Controller.create_player("hasan")

In [ ]:
Controller.player_objects["hasan"].land_objects 
for i in Controller.player_objects.keys():
  for land in Controller.player_objects[i].land_objects.values():
    print(i,land, land.machine_objects.keys())

milad <__main__.Land object at 0x7f7741c377b8> dict_keys([2])
hasan <__main__.Land object at 0x7f7741c37160> dict_keys([2])


END   TESTING CODES ######################################################################









In [ ]:
import numpy as np 
import pandas as pd 
import random

In [188]:
class Controller():
  return_message=""
  turn = 0 
  last_turn = 50
  player_objects = dict()#name,object

  def next_turn():
    turn+=1
    if turn == Controller.last_turn : 
      score_board = Controller.score_board()
      Controller.return_message = "End Game. Thank u " + score_board

  
  def score_board():
    net_worths = pd.Series([])
    for player in list(player_objects.keys()):
      obj = player_objects[player]
      obj.calculate_final_net_worth()
      net_worths[player] = obj 
    net_worths = net_worths.sort(ascending=False)
    return net_worths


  # def get_status(player_name):
    # for land in player_objecs[player_name].

  def create_player(player_name):
    land_id = random.choice(Land.available_lands) #random از بین لیست زمین های در دسترس
    Land.available_lands.remove(land_id)   #remove from available lands
    machine_id =  2 # از دیتافریم زمین و ماشین خوانده میشود
    Controller.player_objects.update({player_name : Player(player_name)})
    Controller.buy_land(player_name, land_id)
    Controller.buy_machine(player_name, land_id, machine_id)
    
  def buy_land(player_name, land_id):
    Controller.player_objects[player_name].add_land(land_id)
    # Controller.player_objects[name].cash -=         اینجا باید پول زمین رو از حسابش کم کنیم . فعلا لازم نیست  چون خودمون بهش زمین رو اختصاص میدیم 

  def buy_machine(player_name, land_id, machine_id):
    Controller.player_objects[player_name].add_machine(land_id, machine_id)

  def start_production(player_name, product_name, amount):# This Func should take machine_id too ( Later Inshallah)
    if not (product_name in Product.products_list):
      return ("wtf ?")
    else : 
      player_obj = Controller.player_objects[player_name]
      stockhouse = list(player_obj.land_objects.values())[0].stockhouse
      if product_name in stockhouse.inventory_df.index:
        stockhouse.inventory_df.loc[product_name, "total_inventory"] += amount
        stockhouse.inventory_df.loc[product_name, "inhand_inventory"] += amount
      else:
        stockhouse.inventory_df.loc[product_name, "total_inventory"] = amount 
        stockhouse.inventory_df.loc[product_name, "inhand_inventory"] = amount 


  def put_into_market(player_name, product_name, amount, price):# it doesn't get stockhouse_id or etc. until we have more than one stockhouse
    product_on_vitrin_id = random.choice([i for i in range(1000) if not(i in list(Market.product_on_vitrin_objects.keys()))])   # a random unique id , that will assign to vitrin objects
    player_obj = Controller.player_objects[player_name]
    stockhouse_obj = list(player_obj.land_objects.values())[0].stockhouse
    

    #check the inventory
    if not (product_name in stockhouse_obj.inventory_df.index):
      return ("Doesn't Exist")
    else:
      if stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] < amount : 
         return("No Enough Inventory")
      else :
        Market.product_on_vitrin_objects.update({product_on_vitrin_id : ProductOnVitrin(player_obj, stockhouse_obj, product_name, amount, price)})
        stockhouse.inventory_df.loc[product_name, "inhand_inventory"] -= amount
        return("Done :)")
    
  
  def upgrade_stockhouse(player_name, land_id):
    pass

  def show_product_market_info():
    return Market.products_on_vitrin

  def buy_product(player_name, product_on_vitrin_id, amount): # آیدی محصول داخل ویترین رو میده که بریم از روی دیکشنری توی مارکت آبجکتش رو پیدا کنیم و انتقال رو انجام بدیم 
    player_obj = Controller.player_objects[player_name]
    stockhouse = list(player_obj.land_objects.values())[0].stockhouse
    product_on_vitrin_obj = Market.product_on_vitrin_objects[product_on_vitrin_id]
    seller_player = product_on_vitrin_obj.seller_obj
    if amount > stockhouser.remaining_cap:
      return ("No Enough Space")
    elif player_obj.cash < product_on_vitrin_obj.price * amount :
      return ("No Enough Cash")
    elif amount > product_obj.amount :
      return ("No Enough Product In Vitrin")
    else:
      stockhouse.add(_, amount) #add to inventory  # هنوز معلوم نیست چجوری قراره پروداکت رو شناسایی کنیم . بی هویته 
      player_obj.cash -= amount * product_on_vitrin_obj.price #update cash
      seller_stockhouse = list(seller_player.land_objects.values())[0].stockhouse
      seller_player.cash += amount*price
      seller_stockhouse.remove(product_id, amount) # removing selled products from origin stockhouse
      product_on_vitrin_obj.player_obj.cash += amount * product_on_vitrin_obj.price # add cash to seller player
      return("Done :)")
    
   

In [ ]:
class Player():
  players_name=[]

  def __init__(self, player_name):
    self.name = player_name
    #remove from available
    self.cash = 1000000 # بودجه اولیه
    self.land_objects=dict()

  def calculate_final_net_worth(self):
    inventory_value_list = []
    for land in list(self.land_objects.keys()):
      inventory_value_list.append(self.land_objects[land].calculate_inventory_value()) # تو هر خونه این لیست ، ارزش دارایی های یک زمین از زمین های بازیکن ثبت میشه
    inventory_value_sum = sum(inventory_value_list)
    net_worth = inventory_value_sum + self.cash 
    return net_worth 

  def add_land(self, land_id):
    self.land_objects.update({land_id : Land(land_id)}) # یه اینستنس از لند میسازه که ورودی بهش آیدی میدیم . بعد توی خود لند ، بر حسب آیدی بقیه فیچر های لند رو ست میکنیم
    

  def add_machine(self, land_id, machine_id):
    self.land_objects[land_id].add_machine(machine_id)







In [ ]:
class Gamein_Bazaar():

  def put_into_market():
    pass

In [67]:
Land.available_lands

[1, 2, 3, 4, 5, 6]

In [88]:
class Land():
  lands_list =[] #amirreza یه دیتا فریم که یه آی دی زمین داره و ماشین
  available_lands= list(range(100)) #آی دی های باقی مانده
  
  def __init__(self, land_id):
    self.machine_objects = dict()
    self.land_id = land_id
    self.stockhouse = StockHouse()# اینجا دیگه دیکشنری نیست چون فرض بر اینه که یه دونه انبار بیشتر نداریم 


    # self.stockhouse = StockHouse()
  def add_machine(self, machine_id):
    self.machine_objects.update({machine_id: Machine(machine_id)})

    


  

In [146]:

class StockHouse():
  def __init__(self):
    self.remaining_cap = 3000
    self.inventory_df = pd.DataFrame([], columns= ["total_inventory", "inhand_inventory"]) # indexes = product names 
    self.cost_per_product = 100
  def add(self, product_name, amount):
    self.remaining_cap -= amount
    self.inventory_df[product_name] += amount
  def remove(self, product_name, amount):
    pass 


  

In [ ]:
class Machine():
  machines_list = _

  def __init__(self,machine_id):
    self.production_id = _
    self.price = _ 
    self.enabled = True
    self.cost_per_production = _
    self.working = 0
    self.remaining_production = _
    
  def start_production(self):
    pass
  def stop_production(self):
    pass
  

  

In [105]:
class Product():
  products_list = ["corn","rice","tomato","potato","salad_olvie","gheyme"]
  def __init__(self):
    self.material_needed = _
    self.lower_band = _
    self.upper_band = _

    
  

In [ ]:
class Material():
  Prices = _ #dict (id , price)

In [210]:
class Market():
  product_on_vitrin_objects = dict()
  
  products_on_vitrin = pd.DataFrame([], columns=["Supplier", "Product", "Amount", "Price", "Storehouse_object"])
def add_to_market(player_obj, stockhouse_obj, product_obj, amount, price):
  product_on_vitrin_objects.update({product_on_vitrin_id : ProductOnVitrin(player_obj, stockhouse_obj, amount, price)})
  pass
def remove_from_market():
  pass 

In [ ]:
class ProductOnVitrin():
  def __init__(self, player_obj, stockhouse_obj, product_obj, amount, price):
    self.seller_obj = player_obj
    self.stockhouse_obj = stockhouse_obj
    self.product_obj = product_obj
    self.amount = amount
    self.price = price